In [5]:
!pip freeze | findstr scikit

scikit-image==0.18.3
scikit-learn @ file:///D:/bld/scikit-learn_1640464393983/work


In [6]:
import pickle
import pandas as pd

In [7]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [8]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [10]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [11]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [12]:
y_pred.mean()

16.191691679979066

In [17]:
df_out = pd.DataFrame()
month=2;year=2021
df_out['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_out['preds'] = y_pred
df_out

,ride_id,preds
0,2021/02_1,14.539865
1,2021/02_2,13.740422
2,2021/02_3,15.593339
3,2021/02_4,15.188118
4,2021/02_5,13.817206
...,...,...
990108,2021/02_1037687,12.433246
990109,2021/02_1037688,12.910885
990110,2021/02_1037689,15.834923
990111,2021/02_1037690,16.783176


In [21]:
output_file = "../../../mlops-zoomcamp-data/hmw04/df_out_{year:04d}_{month:02d}.parquet"
df_out.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)